# Spanish Court Tribunals Webscrape

This script scrapes data from on Spanish Court Tribunals from [this](http://www.minhafp.gob.es/es-ES/Areas%20Tematicas/Contratacion/TACRC/Paginas/BuscadordeResoluciones.aspx?pagina=1) website, before putting the results in a table and exporting to CSV.

## Load in require libraries

In [2]:
import requests #For sending GET requests to websites
from lxml import html #For parsing HTML
import pandas as pd #For creating and manipulating dataframes
import re #For handling regular expressions in text
from datetime import datetime #For processing dates
import timeit #For timing executed code

## Create required functions

In [3]:
#if one string (x) is found within another string (z), replace it with a final string (y)
def clean_issue(x,y,z):
    try:
        output = re.sub(x,y,z)
    
    except TypeError:
        output = ''
    
    return output

#If one string is present in a second string, remove the first string from the second string
def remove_text(x,y):
    try:
        output = re.sub(re.escape(x),'',y)
    
    except TypeError:
        output = None
    
    return output

#Shorten outcome type down to certain number of predefined strings based on a longer string's content
def outcome_reduction(long_outcome):
    
    long_outcome = str(long_outcome)
    
    outcome_list = ['Inadmision y desestimacion','Estimacion parcial',
                    'Desestimacion','Inadmision','Estimacion',
                    'Desistimiento del recurrente'] 

    for item in outcome_list:
        if item in long_outcome:
            long_outcome = item
            break
        else:
            continue

    return(long_outcome)

#If one string is present in a second string, remove the first string from the second string - NOT REQUIRED
def extra_outcome_text(string1,string2):
    try:
        diff_string = re.sub(re.escape(string1),'',string2)
    except TypeError:
        diff_string = None
        
    return(diff_string)

#Split up the legislation column by either a full stop or comma as a seperator
def split_legis(string):
    try:
        snipped = re.split('\\.|,',string)[1]
    except IndexError:
        snipped = ''
    return(snipped)

#Get outcomes from text column
def get_outcome(string1):
    try:    
        string2 = string1.split('.')[1]
    except IndexError:
        string2 = 'NaN'
    return(string2)

#Remove accents from words
def remove_accents(my_string):
    my_string = re.sub('á','a',my_string)
    my_string = re.sub('é','e',my_string)
    my_string = re.sub('í','i',my_string)  
    my_string = re.sub('ó','o',my_string)
    my_string = re.sub('º','o',my_string)
    my_string = re.sub('ú','u',my_string)
    return(my_string)

#Shorten outcome down to certain number of predefined strings based on a longer string's content
def id_outcome_type(outcome_string):
    global contract_type
    if 'inadmision y desestimacion' in outcome_string:
        contract_type = 'Inadmision y Desestimacion'
    elif 'estimacion parcial' in outcome_string:
        contract_type = 'Estimacion Parcial'
    elif 'desestimacion' in outcome_string:
        contract_type = 'Desestimacion'
    elif 'inadmision' in outcome_string:
        contract_type = 'Inadmision'
    elif 'estimacion' in outcome_string:
        contract_type = 'Estimacion'
    elif 'desestimiento del recurrente' in outcome_string:
        contract_type = 'Desestimiento del Recurrente'
    else:
        contract_type = 'NaN'
    return(contract_type)

#Shorten contract type down to certain number of predefined strings based on a longer string's content
def id_contract_type(issue_string):
    global contract_type
    if 'gestion de servicios publicos' in issue_string or 'gestion de servicio publico' in issue_string:
        contract_type = 'Gestion de servicios publicos'
    elif ('servicios' in issue_string and 'suministros' in issue_string) and 'gestion de servicios publicos' not in issue_string:
        contract_type = 'Servicios y suministro'
    elif ('servicio' in issue_string or 'servicios' in issue_string) and 'gestion de servicios publicos' not in issue_string:
        contract_type = 'Servicios'
    elif 'gestion de servicios publicos' in issue_string:
        contract_type = 'gestion de servicios publicos'
    elif 'suministro' in issue_string:
        contract_type = 'Suministro'
    elif 'obras' in issue_string:
        contract_type = 'Obras'
    elif 'concesion de obra publica' in issue_string:
        contract_type = 'Concesion de obra publica'
    elif 'otros' in issue_string:
        contract_type = 'Otros'
    else:
        contract_type = 'NaN'

    return(contract_type)

#Shorten issue down to certain number of predefined strings based on a longer string's content
def id_issue_type(issue_string):
    global issue_type
    if ('Adjudicacion' in issue_string or 'adjudicacion' in issue_string) and ('Exclusion' in issue_string or 'exclusion') in issue_string:
        issue_type = 'Adjudicacion y exclusion'
    if ('Pliegos' in issue_string or 'pliegos' in issue_string) and ('Adjudicacion' in issue_string or 'adjudicacion') in issue_string:
        issue_type = 'Pliegos y adjudicacion'
    if ('Pliegos' in issue_string or 'pliegos' in issue_string) and ('Anuncio' in issue_string or 'anuncio') in issue_string:
        issue_type = 'Pliegos y anuncio'
    if 'Adjudicacion' in issue_string or 'adjudicacion' in issue_string:
        issue_type = 'Adjudicacion'
    elif 'Pliegos' in issue_string or 'pliegos' in issue_string or 'lospliegos' in issue_string or 'PPT' in issue_string or 'PCAP' in issue_string:
        issue_type = 'Pliegos'
    elif 'Criterios de valoracion' in issue_string or 'criterios de valoracion' in issue_string:
        issue_type = 'Criterios de valoracion'
    elif 'Licitacion' in issue_string or 'licitacion' in issue_string:
        issue_type = 'Licitacion'
    elif 'anuncio de convocatoria' in issue_string:
        issue_type = 'Anuncio de convocatoria'
    elif 'acto de apertura de sobres' in issue_string:
        issue_type = 'Acto de apertura de sobres'
    elif 'Exclusion' in issue_string or 'exclusion' in issue_string:
        issue_type = 'Exclusion'
    elif 'Desistimiento' in issue_string or 'Desierta' in issue_string or 'Desierto' in issue_string or 'desistimiento' in issue_string or 'desierta' in issue_string or 'desierto' in issue_string:
        issue_type = 'Desistimiento, Desierta, Desierto'
    elif 'Desistimiento' in issue_string or 'Desierta' in issue_string or 'Desierto' in issue_string or 'desistimiento' in issue_string or 'desierta' in issue_string or 'desierto' in issue_string:
        issue_type = 'Desistimiento, Desierta, Desierto'
    else:
        issue_type = 'NaN'

    return(issue_type)    

#Create a boolean column for if the type of issue is Acuerdo Marco or not
def acuerdo_marco(issue_string):
    if 'Acuerdo Marco' in issue_string or 'AM' in issue_string or 'Acuerdo marco' in issue_string:
        issue_type = True
    else:
        issue_type = False

    return(issue_type)

## Identify total number of pages to be scraped

In [4]:
#URL of first page, which contains total number of pages at bottom
url = 'http://www.minhafp.gob.es/es-ES/Areas%20Tematicas/Contratacion/TACRC/Paginas/BuscadordeResoluciones.aspx?pagina=1'

#Send HTTP request to page
page = requests.get(url)

#Parse page response as HTML tree
tree = html.fromstring(page.content)

#Select XPath for number of pages
no_pages = tree.xpath('//*[@id="ctl00_SPWebPartManager1_g_93bc4c3a_0f69_4097_bed1_978c8b545335"]/div/div/div/p[1]/text()')

#Strip out non-required characters and turn into integer
no_pages = int(re.sub(".*\\/","",no_pages[0]))

#Display
no_pages

648

## Run scrape

In [7]:
#Create empty lists to append each page's data into
resolution_number_all = []
size_all = []
date_all = []
issue_all = []
legislation_all = []
text_all = []

#Set a timer for when the scrape starts, to estimate completion
start = timeit.default_timer()

#Loop through each page number between 1 and the total number of pages identified above
for i in range(1,no_pages+1):
    
    #URL of page
    url = 'http://www.minhafp.gob.es/es-ES/Areas%20Tematicas/Contratacion/TACRC/Paginas/BuscadordeResoluciones.aspx?pagina='+str(i)
    #Send HTTP request to page
    page = requests.get(url)
    #Parse page response as HTML tree
    tree = html.fromstring(page.content)
    
    #Resolution numbers
    resolution_number = tree.xpath("//div[contains(@class, 'breaker_resultados')]/a/text()")
    resolution_number = [re.sub("\\s+","",item) for item in resolution_number]
    resolution_number_all = resolution_number_all + resolution_number
    
    #Size of file
    size = tree.xpath("//ul[contains(@class, 'listaResultadoBusqueda')]/li/div[2]/text()")
    size = [re.sub("Kb.*","",item) for item in size]
    size = [re.sub(".*pdf","",item) for item in size]
    size = [re.sub(".*doc","",item) for item in size]
    size = [re.sub("\\s+","",item) for item in size]
    size = [re.sub("-","",item) for item in size]
    size = [re.sub(",",".",item) for item in size]
    size = [float(item) for item in size]
    size_all = size_all + size
    
    #Date of resolution
    date = tree.xpath("//ul[contains(@class, 'listaResultadoBusqueda')]/li/div[4]/text()[2]")
    date = [re.sub("\\s+","",item) for item in date]
    date = [re.sub(":","",item) for item in date]
    date = [datetime.strptime(item, '%d/%m/%Y') for item in date]
    date_all = date_all + date
    
    #Issue raised by complainant
    issue = tree.xpath("//ul[contains(@class, 'listaResultadoBusqueda')]/li/div[6]/text()[2]")
    issue = [re.sub("\n"," ",item) for item in issue]
    issue = [re.sub(",.*","",item) for item in issue]
    issue = [re.sub("\\s\\s+","",item) for item in issue]
    issue = [re.sub(":","",item) for item in issue]
    issue_all = issue_all + issue
    
    #Legislation applied to test case
    legislation = tree.xpath("//ul[contains(@class, 'listaResultadoBusqueda')]/li/div[6]/text()[2]")
    legislation = [re.sub(":\r\\s+"," ",item) for item in legislation]
    legislation = [re.sub("\n"," ",item) for item in legislation]
    legislation = [split_legis(item) for item in legislation] 
    legislation = [re.sub("\\s","",item) for item in legislation]
    legislation_all = legislation_all + legislation
    
    #Free text associated with resolution
    text = tree.xpath("//ul[contains(@class, 'listaResultadoBusqueda')]/li/div[6]/text()[2]")
    #text = [re.sub("\n"," ",item) for item in text]
    text_all = text_all + text
    
    #Record time
    stop = timeit.default_timer()
    print(str(i),
          '| Time Elapsed',
          str(round((stop - start)/60,2)),
         'mins | Estimated Completion: ' + str(round((stop - start)/i * no_pages/60,2)),
         'mins')

#Convert to a dataframe
tribunals_df = pd.DataFrame({'resolution_number':resolution_number_all,
                             'size': size_all,
                             'date': date_all,
                             'issue': issue_all,
                             'legislation': legislation_all,
                             'text': text_all})

#Select certain columns
tribunals_df = tribunals_df[['date', 'issue', 'legislation','resolution_number', 'size', 'text']]

1 | Time Elapsed 0.02 mins | Estimated Completion: 10.12 mins
2 | Time Elapsed 0.03 mins | Estimated Completion: 8.55 mins
3 | Time Elapsed 0.05 mins | Estimated Completion: 11.85 mins
4 | Time Elapsed 0.09 mins | Estimated Completion: 14.08 mins
5 | Time Elapsed 0.12 mins | Estimated Completion: 15.82 mins
6 | Time Elapsed 0.16 mins | Estimated Completion: 17.06 mins
7 | Time Elapsed 0.19 mins | Estimated Completion: 17.89 mins
8 | Time Elapsed 0.23 mins | Estimated Completion: 18.54 mins
9 | Time Elapsed 0.26 mins | Estimated Completion: 18.82 mins
10 | Time Elapsed 0.28 mins | Estimated Completion: 18.07 mins
11 | Time Elapsed 0.29 mins | Estimated Completion: 16.94 mins
12 | Time Elapsed 0.31 mins | Estimated Completion: 16.48 mins
13 | Time Elapsed 0.33 mins | Estimated Completion: 16.43 mins
14 | Time Elapsed 0.36 mins | Estimated Completion: 16.88 mins
15 | Time Elapsed 0.4 mins | Estimated Completion: 17.1 mins
16 | Time Elapsed 0.43 mins | Estimated Completion: 17.52 mins
17 |

132 | Time Elapsed 1.66 mins | Estimated Completion: 8.13 mins
133 | Time Elapsed 1.67 mins | Estimated Completion: 8.11 mins
134 | Time Elapsed 1.67 mins | Estimated Completion: 8.08 mins
135 | Time Elapsed 1.68 mins | Estimated Completion: 8.05 mins
136 | Time Elapsed 1.68 mins | Estimated Completion: 8.02 mins
137 | Time Elapsed 1.69 mins | Estimated Completion: 8.0 mins
138 | Time Elapsed 1.7 mins | Estimated Completion: 7.97 mins
139 | Time Elapsed 1.71 mins | Estimated Completion: 7.95 mins
140 | Time Elapsed 1.71 mins | Estimated Completion: 7.93 mins
141 | Time Elapsed 1.72 mins | Estimated Completion: 7.91 mins
142 | Time Elapsed 1.73 mins | Estimated Completion: 7.88 mins
143 | Time Elapsed 1.73 mins | Estimated Completion: 7.86 mins
144 | Time Elapsed 1.74 mins | Estimated Completion: 7.85 mins
145 | Time Elapsed 1.75 mins | Estimated Completion: 7.84 mins
146 | Time Elapsed 1.76 mins | Estimated Completion: 7.82 mins
147 | Time Elapsed 1.77 mins | Estimated Completion: 7.82

263 | Time Elapsed 3.82 mins | Estimated Completion: 9.4 mins
264 | Time Elapsed 3.83 mins | Estimated Completion: 9.41 mins
265 | Time Elapsed 3.85 mins | Estimated Completion: 9.41 mins
266 | Time Elapsed 3.87 mins | Estimated Completion: 9.42 mins
267 | Time Elapsed 3.88 mins | Estimated Completion: 9.42 mins
268 | Time Elapsed 3.9 mins | Estimated Completion: 9.42 mins
269 | Time Elapsed 3.92 mins | Estimated Completion: 9.43 mins
270 | Time Elapsed 3.93 mins | Estimated Completion: 9.44 mins
271 | Time Elapsed 3.96 mins | Estimated Completion: 9.46 mins
272 | Time Elapsed 3.98 mins | Estimated Completion: 9.48 mins
273 | Time Elapsed 4.0 mins | Estimated Completion: 9.49 mins
274 | Time Elapsed 4.03 mins | Estimated Completion: 9.52 mins
275 | Time Elapsed 4.05 mins | Estimated Completion: 9.55 mins
276 | Time Elapsed 4.08 mins | Estimated Completion: 9.57 mins
277 | Time Elapsed 4.1 mins | Estimated Completion: 9.6 mins
278 | Time Elapsed 4.12 mins | Estimated Completion: 9.61 mi

393 | Time Elapsed 6.16 mins | Estimated Completion: 10.16 mins
394 | Time Elapsed 6.17 mins | Estimated Completion: 10.16 mins
395 | Time Elapsed 6.19 mins | Estimated Completion: 10.15 mins
396 | Time Elapsed 6.2 mins | Estimated Completion: 10.15 mins
397 | Time Elapsed 6.22 mins | Estimated Completion: 10.15 mins
398 | Time Elapsed 6.24 mins | Estimated Completion: 10.15 mins
399 | Time Elapsed 6.25 mins | Estimated Completion: 10.16 mins
400 | Time Elapsed 6.27 mins | Estimated Completion: 10.16 mins
401 | Time Elapsed 6.3 mins | Estimated Completion: 10.17 mins
402 | Time Elapsed 6.32 mins | Estimated Completion: 10.19 mins
403 | Time Elapsed 6.35 mins | Estimated Completion: 10.21 mins
404 | Time Elapsed 6.38 mins | Estimated Completion: 10.23 mins
405 | Time Elapsed 6.41 mins | Estimated Completion: 10.26 mins
406 | Time Elapsed 6.43 mins | Estimated Completion: 10.26 mins
407 | Time Elapsed 6.44 mins | Estimated Completion: 10.26 mins
408 | Time Elapsed 6.46 mins | Estimated C

522 | Time Elapsed 8.3 mins | Estimated Completion: 10.31 mins
523 | Time Elapsed 8.32 mins | Estimated Completion: 10.31 mins
524 | Time Elapsed 8.34 mins | Estimated Completion: 10.31 mins
525 | Time Elapsed 8.36 mins | Estimated Completion: 10.32 mins
526 | Time Elapsed 8.38 mins | Estimated Completion: 10.32 mins
527 | Time Elapsed 8.4 mins | Estimated Completion: 10.33 mins
528 | Time Elapsed 8.41 mins | Estimated Completion: 10.33 mins
529 | Time Elapsed 8.43 mins | Estimated Completion: 10.33 mins
530 | Time Elapsed 8.45 mins | Estimated Completion: 10.34 mins
531 | Time Elapsed 8.47 mins | Estimated Completion: 10.34 mins
532 | Time Elapsed 8.49 mins | Estimated Completion: 10.34 mins
533 | Time Elapsed 8.51 mins | Estimated Completion: 10.35 mins
534 | Time Elapsed 8.53 mins | Estimated Completion: 10.35 mins
535 | Time Elapsed 8.55 mins | Estimated Completion: 10.35 mins
536 | Time Elapsed 8.56 mins | Estimated Completion: 10.35 mins
537 | Time Elapsed 8.58 mins | Estimated C

## Clean data

In [8]:

#Clean issue column
tribunals_df['issue'] = tribunals_df['issue'].map(lambda x: clean_issue('\r','',x))
tribunals_df['issue'] = tribunals_df['issue'].map(lambda x: clean_issue('\n','',x))

#Clean text column
tribunals_df['text'] = tribunals_df['text'].map(lambda x: re.sub(':\r\n\\s+','',x))
tribunals_df['text'] = tribunals_df['text'].map(lambda x: re.sub('\r','',x))
tribunals_df['text'] = tribunals_df['text'].map(lambda x: re.sub('\\s\\s+','',x))
tribunals_df['text'] = tribunals_df['text'].map(lambda x: re.sub('\n','',x))

#Remove issue text from free text column
tribunals_df['text'] = tribunals_df.apply(lambda df: remove_text(df['issue'],df['text']),axis=1)

#Limit legislation to a number of categories
tribunals_df['legislation'] = pd.Categorical(tribunals_df['legislation'], 
                                             categories=['TRLCSP','LCSE','LCSPDS','UTE','TRCLSP','TRLCS','TRLCPS',
                                                        'TRLSCP','LCSP','SA','LCSEP','PN','RLCSP','SA(SEGITTUR)',
                                                        'TRLCSPE','TRLKCSP','SA(TRAGSA)','TRLCSPDS','LCSPD',
                                                        'LCSSPDS'], ordered=False)

#Remove legislation text from free text column to form outcome of the resolution
tribunals_df['outcome'] = tribunals_df.apply(lambda df: remove_text(df['legislation'],df['text']),axis=1)

#Clean outcome column
tribunals_df['outcome'] = tribunals_df['outcome'].map(lambda x: re.sub('\\s\\.',' . ',str(x)))
tribunals_df['outcome'] = tribunals_df['outcome'].map(lambda x: re.sub('^,\\s\\.\\s','',x))
tribunals_df['outcome'] = tribunals_df['outcome'].map(lambda x: re.sub('\\..*','',x))

#Clean text column
tribunals_df['text'] = tribunals_df['text'].map(lambda x: re.sub('\\.','. ',x))

#Create temporary second outcome column which reduces outcome to a number of categories
tribunals_df['outcome2'] = tribunals_df['outcome'].map(lambda x: outcome_reduction(x))

#Remove strings not required from outcome column
tribunals_df['outcome_text'] = tribunals_df.apply(lambda df: remove_text(df['outcome2'],df['outcome']),axis=1)

#Any blank outcomes are set to NaN
tribunals_df.loc[tribunals_df['outcome'].str.len() == 0, 'outcome'] = "NaN"

#Clean up issue column
tribunals_df['text2'] = tribunals_df['issue'].map(lambda x: str(x))
tribunals_df['text2'] = tribunals_df['text2'].map(lambda x: re.sub('\\.[0-9]\\.-',', ',x))
tribunals_df['text2'] = tribunals_df['text2'].map(lambda x: re.sub('[0-9]\\.-','',x))
tribunals_df['issue'] = tribunals_df['text2'].map(lambda x: re.sub('\\..*','',x))
tribunals_df['issue'] = tribunals_df['issue'].map(lambda x: re.sub('^\\s','',x))

#Select first item in outcome column to clean up
tribunals_df['outcome2'] = tribunals_df['text'].map(lambda x: get_outcome(x))

#Remove accents
tribunals_df['outcome2'] = tribunals_df['outcome2'].map(lambda x: remove_accents(x))

#Correct spelling mistake
tribunals_df['outcome2'][tribunals_df['outcome2'] == 'Inadmisisón'] = 'Inadmisión'

#Set any cells in temporary outcome column which are NaN to NaN in permanent column
tribunals_df['outcome'][tribunals_df['outcome'] == 'NaN'] = tribunals_df['outcome2'][tribunals_df['outcome'] == 'NaN']

#Turn temporary outcome column to lowercase for ease of manipulation
tribunals_df['outcome2'] = tribunals_df['outcome2'].str.lower()

#Clean outcomes
tribunals_df['outcome2'] = tribunals_df['outcome2'].map(lambda x: id_outcome_type(x))

tribunals_df['outcome'] = pd.Categorical(tribunals_df['outcome2'], 
                                             categories=['Inadmision y Desestimacion','Estimacion Parcial',
                                                            'Desestimacion','Inadmision','Estimacion',
                                                        'Desistimiento del Recurrente', 'NaN'],
                                          ordered=False)

#Clean text column further, including removing any text already in legislation or outcome columns
tribunals_df['text'] = tribunals_df.apply(lambda df: remove_text(df['legislation'],df['text']),axis=1)
tribunals_df['text'] = tribunals_df['text'].map(lambda x: re.sub('^,\\s\\.\\s','',str(x)))
tribunals_df['text'] = tribunals_df.apply(lambda df: remove_text(df['outcome'],df['text']),axis=1)
tribunals_df['text'] = tribunals_df['text'].map(lambda x: re.sub('^\\.\\s','',str(x)))

#Set any empty text cells to NaN
tribunals_df.loc[tribunals_df['text'].str.len() == 0, 'text'] = "NaN"

#Remove defunct columns
tribunals_df = tribunals_df[['resolution_number','date','size','issue','legislation','outcome','text']]

#Clean outcome column further, removing superfluous text
tribunals_df['outcome2'] = tribunals_df['outcome'].map(lambda x: outcome_reduction(x))
tribunals_df['outcome_text'] = tribunals_df.apply(lambda df: remove_text(df['outcome2'],df['outcome']),axis=1)

#Remove defunct columns
tribunals_df = tribunals_df[['resolution_number','date','size','issue','legislation','outcome','text']]

#Remove accents
tribunals_df['issue'] = tribunals_df['issue'].map(lambda x: remove_accents(x))
tribunals_df['text'] = tribunals_df['text'].map(lambda x: remove_accents(x))

#Clean contract type
tribunals_df['contract'] = tribunals_df['issue'].map(lambda x: id_contract_type(x))

#Create Acuerdo Marco column
tribunals_df['acuerdo_marco'] = tribunals_df['issue'].map(lambda x: acuerdo_marco(x))

#Clean issue type column
tribunals_df['issue'] = tribunals_df['issue'].map(lambda x: id_issue_type(x))

tribunals_df

#Output to CSV
#tribunals_df.to_csv("../output/tribunals.csv", encoding="utf-8")

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,resolution_number,date,size,issue,legislation,outcome,text,contract,acuerdo_marco
0,0862/2018,2018-10-01,145.89,Pliegos,LCSP,Inadmision,Inadmision. Falta de legitimacion del recurr...,Servicios,False
1,0812/2018,2018-09-14,180.20,Adjudicacion,LCSP,Estimacion,Estimacion. La exclusion por anormalidad de ...,Servicios,False
2,0809/2018,2018-09-14,91.59,Adjudicacion,TRLCSP,Inadmision,Inadmision. El objeto del contrato esta incl...,Servicios,False
3,0798/2018,2018-09-14,182.61,Pliegos,LCSP,Estimacion,Estimacion. Contra clausula 2. 8. 2 Pliegos ...,Servicios,False
4,0810/2018,2018-09-14,165.64,Exclusion,LCSP,NaN,", que eran confusos, por lo que no se consider...",Servicios,False
5,0808/2018,2018-09-14,225.37,Adjudicacion,LCSP,Estimacion,Estimacion. Se impugna la adjudicacion efect...,Servicios,False
6,0799/2018,2018-09-14,114.70,Exclusion,LCSP,NaN,NaN,Servicios,False
7,0790/2018,2018-09-14,141.87,Pliegos,LCSP,Inadmision,Inadmision. Contratacion del transporte con ...,Servicios,False
8,0791/2018,2018-09-14,181.03,Exclusion,LCSP,NaN,NaN,Servicios,False
9,0796/2018,2018-09-14,163.66,Exclusion,LCSP,NaN,Recurso contra exclusion en contrato de servic...,Servicios,False
